Training Video Classification

**Note:** 
Just run this notebook alone
Setup.ipynb and training videos needs to be changed before running all notebooks

In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import preprocess_input

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from skimage.transform import resize 

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
# read in training frames, create dataframe  
train = pd.read_csv('myVids/train_new.csv')
train

,image,class
0,3_1.mp4_frame6.jpg,1
1,3_1.mp4_frame7.jpg,1
2,3_1.mp4_frame5.jpg,1
3,3_1.mp4_frame4.jpg,1
4,3_1.mp4_frame0.jpg,1
5,3_1.mp4_frame1.jpg,1
6,3_1.mp4_frame3.jpg,1
7,3_1.mp4_frame2.jpg,1
8,2_0.mp4_frame11.jpg,0
9,2_0.mp4_frame3.jpg,0


### What is this dataframe showing? 

This dataframe called "train" has two columns: "image" and "class"
- "image": contains names of each frame 
- "class": contains corresponding class for each frame

##### What does class numbers mean?
0 for good serve   
1 for bad serve 

In this case, **video 1 and 3** is a bad serve and **video 2 and 4** are good serves.


In [3]:
train_image = []

for i in tqdm(range(train.shape[0])):
    # loading the image and resizing to (224,224,3), the input required for vgg16
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 37/37 [00:01<00:00, 18.73it/s]


(37, 224, 224, 3)

We have 37 images each of size 224, 224 and 3. Next we will create a validation set. 

We need to do **two** things to train our model: 
- train images
- their corresponding model 

In [4]:
y = train["class"]
print(train['class'])

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
Name: class, dtype: int64


### One hot Encoding Classes 

Since we have 2 classes: 0 and 1 (which are numerical data) we need to convert to categorical

[Click me for more info on one hot encoding](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/)

In [5]:
dummy_y = to_categorical(y)    # one hot encoding Classes

### VGG16 parameters

VGG16 takes an input shape of (224 x 224 x 3) 

Our images are all different sizes so let's reshape them using resize() function. 

In [6]:
image = []
for i in range(0,X.shape[0]): 
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

### Preprocess 

This function preprocess our images so that it fits within the VGG16 model 

In [7]:
X = preprocess_input(X, mode='tf')      # preprocessing the input dat

### Split ya models 
Why do we need to split our models? Because right now we only have training images, we need to have a validation set to check the performance of models on unseen images 

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)    # preparing the validation set

### Let's build the model from VGG16

In [9]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 


Instructions for updating:
Colocations handled automatically by placer.


### Why include_top = False

Fully connected layers at the end can only take fixed size inputs which was probably where we were going wrong 

In [10]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

((25, 7, 7, 512), (12, 7, 7, 512))

Shape into single dimensions by below: 

In [11]:
X_train = X_train.reshape(25, 7*7*512)
X_valid = X_valid.reshape(12, 7*7*512)

In [12]:
#normalize me 
train = X_train/X_train.max()      
X_valid = X_valid/X_train.max()

### Steps to setting up model 
1. Build model 
2. Compile model 
3. Train model 

In [73]:
model = Sequential()
model.add(InputLayer((28*3,)))    # input layer
model.add(Dense(units=10, activation='sigmoid')) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

In [74]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 10)                850       
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 22        
Total params: 872
Trainable params: 872
Non-trainable params: 0
_________________________________________________________________


### What does model.summary() show us? 

We have a hidden layer with 10 neurons and output layer with 2 neurons(3 classes to predict) 

In [75]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [76]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

<li>An epoch is a measure of the number of times all of the training vectors are used once to update the weights. One epoch is equal to one forward pass and one backward pass of all the training material. </li>
<li>The batch size defines the number of samples that will be propagated through the network in one forward/backward pass. We want to use a batch size less than the number of all samples because that requires less memory and trains faster, but we don't want to use a batch size too small because that would lead to a less accurate estimate of the gradient. </li>

In [77]:
model.fit(train, y_train, epochs=10, validation_data=(X_valid, y_valid), callbacks=[mcp_save], batch_size=3)

ValueError: Error when checking input: expected input_14 to have shape (84,) but got array with shape (25088,)

### What does this show us? 

Accuray of 76%

Why so low? Because setup.ipynb needs to be changed because videos are all "good"